# Chapter 8: Dimensionality Reduction

## 8.1 The Curse of Dimensionality

## 8.2 Main Approaches for Dimensionality Reduction

### 8.2.1 Projection

### 8.2.2 Manifold Learning

## 8.3 PCA

### 8.3.1 Preserving the Variance

### 8.3.2 Principal Components

### 8.3.3 Projecting Down to d Dimensions

### 8.3.4 Using Scikit-Learn

### 8.3.5 Explained Variance Ratio

### 8.3.6 Choosing the Right Number of Dimensions

### 8.3.7 PCA for Compression

### 8.3.8 Randomized PCA

### 8.3.9 Incremental PCA

## 8.4 Kernel PCA

### 8.4.1 Selecting a Kernel and Tuning Hyperparameters

## 8.5 LLE

## 8.6 Other Dimensionality Reduction Techniques